## Intro to language modeling: building makemore
### Part 2: MLP

From YouTube video: https://www.youtube.com/watch?v=TCH_1BHY58I by @AndrejKarpathy

#### Library imports

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

#### Dataset import

In [2]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [3]:
# ---------- Multilayer Perceptron ---------- #

In [4]:
# build the vocab of characters and mappings to int
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset

def build_dataset(words):
    block_size = 3 # context len: 
    # how many chars do we take to predict the next one?

    X, Y = [], []
    for w in words:

        # print(w)
        context = [0] * block_size # padded
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '---->', itos[ix])
            context = context[1:] + [ix] # crop & append
            # rolling window of context

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2]) # 3202 for validation
Xte, Yte = build_dataset(words[n2:]) # 3204 for test

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
# X.shape, X.dtype, Y.shape, Y.dtype

In [7]:
len(words), n1, n2

(32033, 25626, 28829)

In [8]:
# look up table C
C = torch.randn((27,2))

In [9]:
# C[5]
# and 
# F.one_hot(torch.tensor(5), num_classes=27).float() @ C
# are the same

In [10]:
# embedding
emb = C[Xtr]
emb.shape

torch.Size([182625, 3, 2])

In [11]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
# first layer
W1 = torch.randn((6,300), generator=g) # 300 neurons
b1 = torch.randn(300, generator=g) # 300 biases
# second layer
W2 = torch.randn((300,27), generator=g)
b2 = torch.randn(27, generator=g)

parameters = [C, W1, b1, W2, b2]

In [12]:
sum(p.nelement() for p in parameters) # num of para in total

10281

In [13]:
h = torch.tanh(emb.view(emb.shape[0],6) @ W1 + b1)
# use emb.shape[0] or -1

# broadcasting b1 to emb.view(emb.shape[0],6) @ W1
# -1~1

In [14]:
# # the size doesn't matter if I use cat & unbind
# # along 1st dimension
# torch.cat(torch.unbind(emb, 1), 1).shape 
# # new memory is created, not efficient enough

# arr = torch.arange(18)
# arr.view(3,3,2)

In [15]:
logits = h @ W2 + b2
# counts = logits.exp()
# prob = counts / counts.sum(1,keepdims=True)
# loss = -prob[torch.arange(32), Y].log().mean() # ideally the nums in prob[torch.arange(32), Y] will be 1
loss = F.cross_entropy(logits, Ytr)
loss

tensor(24.9808)

In [16]:
for p in parameters:
    p.requires_grad = True

In [17]:
# lre: learning rate exponent; lrs: lr search
lre = torch.linspace(-3, 0, 1000)
lrs = 10 ** lre

In [18]:
lri = []
lossi = []
stepi = []

for i in range(10000):
    
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # forward pass
    emb = C[Xtr[ix]] # (32,3,2)
    h = torch.tanh(emb.view(-1,6) @ W1 + b1) # (32,100)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    # lr = lrs[i]
    lr = 10 ** (-0.75)
    for p in parameters:
        p.data += -lr * p.grad
        
    # track stats
#     lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.item())
    

print(loss.item())
# overfitting a single batch of the data and getting low loss

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(12,6), dpi=300)
plt.plot(stepi, lossi)
# from the graph, when the exponent is -1 or -0.9
# it reaches a good learning rate

In [ ]:
emb = C[Xtr] # (32,3,2)
h = torch.tanh(emb.view(-1,6) @ W1 + b1) # (32,100)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
# training loss ~ dev loss --> small nn --> underfitting

In [ ]:
emb = C[Xdev] # (32,3,2)
h = torch.tanh(emb.view(-1,6) @ W1 + b1) # (32,100)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
# training split, dev/validation split, test split
# 80% (param), 10% (hyper param), 10% (performance)